<a href="https://colab.research.google.com/github/vladargunov/CreditDerivativesSimulator/blob/v11/templates/notebooks/template_notebook_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template for Developing Credit Trading Strategies

This notebook provides an environment and some examples how to backtest the trading strategies which will be developed during the CMF project 'Trading Credit Derivatives'.

The GitHub repo can be found [here](https://github.com/vladargunov/CreditDerivativesSimulator).

The main idea of the project is the implementation of the simulator class that takes care of daily backtesting of any customly developed strategy. This notebook is aimed for a release version 1.0.0 and the functionality might change in the future, while for now it is assumed that a trader receives data prior to '2013-01-02' and then trades until '2022-09-22' at daily frequency. Let us see how any user can develop his or her custom trading strategies.


---

Firstly, please set a version of a simulator for such notebook, here it is advisable to use "v1.0".

In [ ]:
VERSION =  "v1.0" #@param {type:"string"}

In [ ]:
#@title [RUN] Install required python libraries and clone the repo
!pip install wandb -q
!git clone -b $VERSION https://github.com/vladargunov/CreditDerivativesSimulator.git

# Import necessary classses
%cd CreditDerivativesSimulator
from src.base_strategy import BaseStrategy
from src.simulator import Simulator

# Import your additional libraries

In [ ]:
# import sklearn
# import statsmodels

# How to create your custom strategy

Any strategy must inherit from the BaseStrategy class, which is needed for correct execution of the simulator.

Any strategy MUST implement two methods, train_model and trade:

- train_model() method is used for training any of your models on the train set, which is defined to be any data before '2013-01-02'. See examples for possible use of it.

- trade() method defines the strategy which will be performed during the testing stage. At each step the strategy receives a dictionary of prices, and must return a dictionary of weights, where each key represents a name of the stock and value represents share of the portfolio in it. Also see examples for possible use.

In [ ]:
"""
The simplest strategy: invest each day 0.1% of your portfolio into spx
and the rest into cash. No prior training is done
"""

class MyFirstStrategy(BaseStrategy):
  def train_model(self, train_data):
    pass

  def trade(self, daily_data) -> dict:
    return {'spx' : .1}

# How to test you strategy

In [ ]:
# Create an instance of your strategy and of simulator
my_strategy = MyFirstStrategy()

# Define a name of your strategy to be represented in wandb
# Also set flags if you wish to log the results into wandb
# or you wand to debug the strategy, in that case only one testing day
# will be executed

# If you use wandb, you will need to past an API key from your
# wandb account
sim = Simulator(use_wandb=True, 
                debug_mode=False, 
                run_name='MyFirstStrategy1')

sim.simulate(strategy=my_strategy)

# Sharpe ratio is calculated as a mean of daily returns
# of a portfolio minus the mean of returns of a portfolio 
# consisting of 100% spx long index, divided by the standard
# deviation of them

In [ ]:
# You can also access the universe of available asset
# by calling this funciton
sim.get_availiable_assets()

# A more advanced strategy

In [ ]:
"""
Cointegration strategy for assets between 'itraxx_main_generic' and 'spx'
"""
import numpy as np
from sklearn.linear_model import LinearRegression

class MySimpleCointegration(BaseStrategy):
  def train_model(self, train_data):
    # Dropna values
    train_data = train_data.diff().dropna()
    # Run simple regression of itraxx_main_generic
    # and spx
    x = train_data['itraxx_main_generic'].to_numpy().reshape(-1, 1)
    y = train_data['spx'].to_numpy().reshape(-1, 1)

    # Save regression as a class attribute to use in trade
    self.reg = LinearRegression().fit(x,y)

    # Extract linear relationship
    # between these two assets
    self.coeff = self.reg.coef_[0]

    # We get spx = beta * itraxx_main_generic,
    # so the risk-neutral portfolio should be
    # {spx : 1 and itraxx_main_generic : - beta}
    # or in other words the weight of 
    # itraxx_main_generic should be - beta times
    # of weight sp


  def trade(self, daily_data) -> dict:
    # In this portfolio we randomize 
    # spx value at .1 or -.1 and 
    # itraxx_main_generic value at
    # min(- spx_weight * beta, .9)
    spx_weight = np.random.choice([.1, -.1])
    return {'spx' : spx_weight, 'itraxx_main_generic' : min( -spx_weight * self.coeff, .9)} 

In [ ]:
# Repeat the testing procedure of a strategy 
# outlined above

my_strategy = MySimpleCointegration()

sim = Simulator(use_wandb=True, 
                debug_mode=False, 
                run_name='MySimpleCointegration1')

sim.simulate(strategy=my_strategy)

# Final Comments

If you find an issue, or wish to ask for a new feature of the simulator, please open an issue, (see a nice [intro](https://www.youtube.com/watch?v=TKJ4RdhyB5Y) how to open them), and I will try to fix it as soon as I can.

Also if you would like to contribute to this simulator as well, I would be more than delighted to work with you. I hope together we will be able to create a decent project that will help us in our professional and personal development :)